In [ ]:
import os
import yaml
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from epynet import Network
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from stable_baselines import DQN
from wdsEnv import wds
from opti_algorithms import nm

import panel as pn
import param
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource as cds

pn.extension()

In [ ]:
def build_plot(junc_coords, pipe_coords, nodal_property):
    data = cds(data={
        'x': junc_coords['x'],
        'y': junc_coords['y'],
        'head': nodal_property
        }
    )

    mapper = linear_cmap(
        field_name = 'head',
        palette = "Viridis11",
        low = min(nodal_property),
        high = max(nodal_property)
    )
    fig = figure()
    edges = fig.line(pipe_coords['x'], pipe_coords['y'])
    nodes = fig.circle(x='x', y='y', color=mapper, source=data, size=12)
    fig = pn.pane.Bokeh(fig, width=600, height=500)
    return fig

In [ ]:
class load_environment(param.Parameterized):
    sel_wds = param.ObjectSelector(default="Anytown", objects=["Anytown", "D-Town"])
    sel_dmd = param.ObjectSelector(default="Original demands", objects=['Original demands', 'Randomized demands'])
    sel_spd = param.ObjectSelector(default="Original speeds", objects=['Original speeds', 'Randomized speeds'])
    act_load = param.Action(lambda x: x.param.trigger('act_load'), label='Load water distribution system')

    def _assemble_junc_coordinates(self, wds):
        junc_x = []
        junc_y = []
        junc_z = []
        for junc in wds.junctions:
            junc_x.append(junc.coordinates[0])
            junc_y.append(junc.coordinates[1])
            junc_z.append(junc.elevation)
        return {'x': junc_x, 'y': junc_y, 'z': junc_z}
    
    def _assemble_pipe_coords(self, wds):
        pipe_x = []
        pipe_y = []
        pipe_z = []
        for pipe in wds.pipes:
            if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
                pipe_x.append(pipe.from_node.coordinates[0])
                pipe_x.append(pipe.to_node.coordinates[0])
                pipe_x.append(float('nan'))

                pipe_y.append(pipe.from_node.coordinates[1])
                pipe_y.append(pipe.to_node.coordinates[1])
                pipe_y.append(float('nan'))

                pipe_z.append(pipe.from_node.elevation)
                pipe_z.append(pipe.to_node.elevation)
                pipe_z.append(float('nan'))
        return {'x': pipe_x, 'y': pipe_y, 'z': pipe_z}

    def load_env(self, wds_name, resetOrigDemands, resetOrigPumpSpeeds):
        if wds_name == 'Anytown':
            hyperparams_fn  = 'anytownMaster'
            model_fn        = 'anytownHO1-best'
        elif wds_name == 'D-Town':
            hyperparams_fn  = 'dtownMaster'
            model_fn        = 'dtownHO1-best'
        else:
            hyperparams_fn  = 'anytownMaster'
            model_fn        = 'anytownHO1-best'

        pathToParams= os.path.join('experiments', 'hyperparameters', hyperparams_fn+'.yaml')
        with open(pathToParams, 'r') as fin:
            hparams = yaml.load(fin, Loader=yaml.Loader)
        self.pathToModel = os.path.join('experiments', 'models', model_fn+'.zip')

        self.env = wds(
                wds_name        = hparams['env']['waterNet']+'_master',
                speed_increment = hparams['env']['speedIncrement'],
                episode_len     = hparams['env']['episodeLen'],
                pump_groups     = hparams['env']['pumpGroups'],
                total_demand_lo = hparams['env']['totalDemandLo'],
                total_demand_hi = hparams['env']['totalDemandHi'],
                reset_orig_pump_speeds  = resetOrigPumpSpeeds,
                reset_orig_demands      = resetOrigDemands
        )
        self.junc_coords = self._assemble_junc_coordinates(self.env.wds)
        self.pipe_coords = self._assemble_pipe_coords(self.env.wds)

    @param.depends('act_load')
    def load_wds(self):
        self.load_env(
            self.sel_wds,
            self.sel_dmd == 'Original demands',
            self.sel_spd == 'Original speeds'
        )
        self.env.reset(training=True)
        self.plot = build_plot(self.junc_coords, self.pipe_coords, self.env.wds.junctions.basedemand)
        return self.plot

In [ ]:
class optimize_speeds(param.Parameterized):
    act_opti = param.Action(lambda x: x.param.trigger('act_opti'), label='Optimize pump speeds')
    
    def __init__(self, environment):
        self.env = environment

In [ ]:
wrapper = load_environment()
pn.Column(
    '# Loading the water distribution system',
    pn.Row(
        pn.Column(pn.panel(wrapper.param, show_labels=False, show_name=False, margin=0,
                           widgets = {
                               'sel_dmd': pn.widgets.RadioButtonGroup,
                               'sel_spd': pn.widgets.RadioButtonGroup
                           }),
        ),
        wrapper.load_wds
    )
).servable()

In [ ]:
model   = DQN.load(wrapper.pathToModel)

obs = wrapper.env.reset()
while not wrapper.env.done:
    act, _              = model.predict(obs, deterministic=True)
    obs, reward, _, _   = wrapper.env.step(act, training=False)
    print(reward)

In [ ]:
obs = wrapper.env.reset()
while not wrapper.env.done:
    act, _              = model.predict(obs, deterministic=True)
    obs, reward, _, _   = wrapper.env.step(act, training=False)
    print(reward)

In [ ]:
speeds, target_val, n_steps   = nm.minimize(wrapper.env.reward_to_scipy, wrapper.env.dimensions)

In [ ]:
wrapper.env.get_state_value_to_opti(speeds // 0.05 * 0.05)

In [ ]:
n_steps

In [ ]:
alma = wrapper.env.wds.junctions.basedemand

In [ ]:
import numpy as np

In [ ]:
np.max(alma)

In [ ]:
np.min(alma)

In [ ]:
np.mean(alma)